# Predicting the Truckee River Flow

https://images.fineartamerica.com/images/artworkimages/mediumlarge/1/fall-on-the-truckee-river-vance-fox.jpg![image.png](attachment:image.png)


The Truckee River relies on snow that falls in the Sierra Nevada during the winter months, and natural run-off combined with releases of water from Lake Tahoe and other reservoirs during the summer months to maintain flows that support water consumption needs.

https://tmwa.com/images/riversystem_map.gif![image.png](attachment:image.png)

## Collecting data